In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# Load dataset
df_2_class = pd.read_csv("/content/drive/My Drive/Colab Notebooks/user_fake_authentic_2class.csv")

Mounted at /content/drive


In [4]:
df_4_class=pd.read_csv("/content/drive/My Drive/Colab Notebooks/user_fake_authentic_4class.csv")

In [5]:
df_2_class.head()

,pos,flw,flg,bl,pic,lin,cl,cz,ni,erl,erc,lt,hc,pr,fo,cs,pi,class
0,44,48,325,33,1,0,12,0.0,0.0,0.00,0.00,0.000,0.0,0.0,0.000,0.111111,0.094985,f
1,10,66,321,150,1,0,213,0.0,1.0,14.39,1.97,0.000,1.5,0.0,0.000,0.206826,230.412857,f
2,33,970,308,101,1,1,436,0.0,1.0,10.10,0.30,0.000,2.5,0.0,0.056,0.572174,43.569939,f
3,70,86,360,14,1,0,0,1.0,0.0,0.78,0.06,0.000,0.0,0.0,0.000,1.000000,5.859799,f
4,3,21,285,73,1,0,93,0.0,0.0,14.29,0.00,0.667,0.0,0.0,0.000,0.300494,0.126019,f


In [6]:
df_4_class.head()

,pos,flw,flg,bl,pic,lin,cl,cz,ni,erl,erc,lt,hc,pr,fo,cs,pi,class
0,69,541,440,293,1,0,223,0.000000,0.111,10.120000,0.50,0.944,0.556,0.0,0.0,0.107055,334.596802,r
1,9,783,803,76,1,0,101,0.000000,0.000,19.820000,1.99,0.889,0.222,0.0,0.0,0.007400,2137.979248,r
2,390,1200,925,103,1,0,723,0.000000,0.000,11.540000,0.76,1.000,2.389,0.0,0.0,0.221954,96.244957,r
3,138,1100,1000,40,1,0,24,0.166667,0.167,20.440001,1.61,0.833,0.111,0.0,0.0,0.025055,226.622437,r
4,1,228,487,84,1,0,30,0.000000,0.000,9.210000,1.75,1.000,0.000,0.0,0.0,0.000000,0.000000,r


In [7]:
df_2_class.shape,df_4_class.shape

((65326, 18), (43307, 18))

In [8]:
df_2_class["class"].unique(),df_4_class["class"].unique()

(array(['f', 'r'], dtype=object), array(['r', 's', 'i', 'a'], dtype=object))

### Change Categorical values in the target to Numerical

In [11]:
encoder=LabelEncoder()
df_2_class["class"]=encoder.fit_transform(df_2_class["class"])
df_4_class["class"]=encoder.fit_transform(df_4_class["class"])

In [12]:
df_2_class["class"].unique(), df_4_class["class"].unique()

(array([0, 1]), array([2, 3, 1, 0]))

In [13]:
X1=df_2_class[df_2_class.columns[:-1]].values
y1=df_2_class[df_2_class.columns[-1]].values

In [14]:
X2=df_4_class[df_4_class.columns[:-1]].values
y2=df_4_class[df_4_class.columns[-1]].values

In [15]:
X1.shape, y1.shape,X2.shape, y2.shape

((65326, 17), (65326,), (43307, 17), (43307,))

### Noirmalize the values

In [16]:
scaler = StandardScaler()
X1 = scaler.fit_transform(X1)
data = np.hstack((X1, np.reshape(y1, (-1, 1))))
transformed_df_2_class = pd.DataFrame(data, columns=df_2_class.columns)

In [17]:
scaler_2 = StandardScaler()
X2 = scaler_2.fit_transform(X2)
data_2 = np.hstack((X2, np.reshape(y2, (-1, 1))))
transformed_df_4_class = pd.DataFrame(data_2, columns=df_4_class.columns)

### Ensure the data is balanced

In [18]:
over = RandomOverSampler()
X1, y1 = over.fit_resample(X1, y1)
data = np.hstack((X1, np.reshape(y1, (-1, 1))))
transformed_df_2_class = pd.DataFrame(data, columns=df_2_class.columns)

In [19]:
over_2 = RandomOverSampler()
X2, y2 = over.fit_resample(X2, y2)
data_2 = np.hstack((X2, np.reshape(y2, (-1, 1))))
transformed_df_4_class = pd.DataFrame(data_2, columns=df_4_class.columns)

In [39]:
transformed_df_4_class["class"].unique()

array([2., 3., 1., 0.])

In [20]:
len(transformed_df_2_class[transformed_df_2_class["class"]==1]), len(transformed_df_2_class[transformed_df_2_class["class"]==0])

(32866, 32866)

In [21]:
len(transformed_df_4_class[transformed_df_4_class["class"]==0]), len(transformed_df_4_class[transformed_df_4_class["class"]==1]),len(transformed_df_4_class[transformed_df_4_class["class"]==2]), len(transformed_df_4_class[transformed_df_4_class["class"]==3])

(12054, 12054, 12054, 12054)

### 2 CLASS MODEL

#### SPLIT DATA

In [22]:
X_train, X_temp, y_train, y_temp = train_test_split(X1, y1, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

#### MODEL 2 CLASS

In [23]:
model_2_class = tf.keras.Sequential([
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(1, activation="sigmoid")
])

In [24]:
model_2_class.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [32]:
model_2_class.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
2465/2465 [==============================] - 13s 5ms/step - loss: 0.2824 - accuracy: 0.8750 - val_loss: 0.2898 - val_accuracy: 0.8771
Epoch 2/10
2465/2465 [==============================] - 10s 4ms/step - loss: 0.2806 - accuracy: 0.8760 - val_loss: 0.2851 - val_accuracy: 0.8760
Epoch 3/10
2465/2465 [==============================] - 12s 5ms/step - loss: 0.2789 - accuracy: 0.8763 - val_loss: 0.2829 - val_accuracy: 0.8771
Epoch 4/10
2465/2465 [==============================] - 12s 5ms/step - loss: 0.2767 - accuracy: 0.8776 - val_loss: 0.2806 - val_accuracy: 0.8776
Epoch 5/10
2465/2465 [==============================] - 10s 4ms/step - loss: 0.2754 - accuracy: 0.8773 - val_loss: 0.2808 - val_accuracy: 0.8776
Epoch 6/10
2465/2465 [==============================] - 10s 4ms/step - loss: 0.2734 - accuracy: 0.8789 - val_loss: 0.2828 - val_accuracy: 0.8763
Epoch 7/10
2465/2465 [==============================] - 11s 4ms/step - loss: 0.2719 - accuracy: 0.8795 - val_loss: 0.2773 - val_ac

In [33]:
model_2_class.evaluate(X_test, y_test)

411/411 [==============================] - 1s 2ms/step - loss: 0.2695 - accuracy: 0.8800


[0.2695465385913849, 0.8800486922264099]

### 4 CLASS MODEL

#### SPLIT DATA

In [34]:
X2_train, X2_temp, y2_train, y2_temp = train_test_split(X2, y2, test_size=0.4, random_state=0)
X2_valid, X2_test, y2_valid, y2_test = train_test_split(X2_temp, y2_temp, test_size=0.5, random_state=0)

In [43]:
y2_train

array([2, 1, 3, ..., 1, 1, 2])

#### CHANGE SHAPE OF THE TARGET DATA TO MATCH THAT OF THE OUPUT LAYER

In [47]:
y2_train = tf.one_hot(y2_train, depth=4)
y2_valid = tf.one_hot(y2_valid, depth=4)
y2_test = tf.one_hot(y2_test, depth=4)

### MODEL 4 CLASS

In [48]:
model_4_class = tf.keras.Sequential([
                             tf.keras.layers.Dense(10, activation='relu'),
                             tf.keras.layers.Dense(4, activation="softmax")
])

In [49]:
model_4_class.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [50]:
model_4_class.fit(X2_train, y2_train, batch_size=16, epochs=10, validation_data=(X2_valid, y2_valid))

Epoch 1/10
1809/1809 [==============================] - 9s 4ms/step - loss: 1.0271 - accuracy: 0.5470 - val_loss: 0.8681 - val_accuracy: 0.6436
Epoch 2/10
1809/1809 [==============================] - 6s 4ms/step - loss: 0.8240 - accuracy: 0.6544 - val_loss: 0.7838 - val_accuracy: 0.6829
Epoch 3/10
1809/1809 [==============================] - 7s 4ms/step - loss: 0.7623 - accuracy: 0.6938 - val_loss: 0.7316 - val_accuracy: 0.7150
Epoch 4/10
1809/1809 [==============================] - 7s 4ms/step - loss: 0.7190 - accuracy: 0.7178 - val_loss: 0.6902 - val_accuracy: 0.7364
Epoch 5/10
1809/1809 [==============================] - 10s 5ms/step - loss: 0.6808 - accuracy: 0.7350 - val_loss: 0.6528 - val_accuracy: 0.7520
Epoch 6/10
1809/1809 [==============================] - 6s 3ms/step - loss: 0.6488 - accuracy: 0.7467 - val_loss: 0.6230 - val_accuracy: 0.7602
Epoch 7/10
1809/1809 [==============================] - 7s 4ms/step - loss: 0.6225 - accuracy: 0.7577 - val_loss: 0.6025 - val_accuracy

In [52]:
model_4_class.evaluate(X2_test,y2_test)

302/302 [==============================] - 1s 3ms/step - loss: 0.5836 - accuracy: 0.7693


[0.583609938621521, 0.7692866325378418]